## Imports and Installs

In [2]:
# !pip3 install seaborn --user
# !pip3 install --index-url  http://pyrrepo.int.cdphp.com/pypi/simple --trusted-host pyrrepo.int.cdphp.com --user pyarrow
# !pip3 install lifelines --user
# !pip3 install pandas --upgrade --user
# !pip3 install pyspark --user
# !pip3 install tqdm --user
# !pip3 install cython --user
# !pip3 install dowhy --user
# !pip3 install rpy2 --user
# !python3 setup.py build_ext --inplace

In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext Cython

import data_process, data_functions
import numpy as np
import warnings
import pandas as pd
from pathlib import Path

## Edit Paths
All local paths are paths in cluster disk space. All AWS paths are keys from S3. <br/>
data_path = local folder where data will be downloaded <br/>
temp_path = where downloaded files are usually saved after they are processed <br/>
canary_aws_path = S3 Key of Canary Source file in Step 1 <br/>
canary_local_path = local path where Canary will be saved after pulled from S3 in Step 1 <br/>
canary_new_aws_path = S3 Key of Canary New Source file in Step 1 <br/>
canary_new_local_path = local path where Canary New will be saved after pulled from S3 in Step 1 <br/>
canary_processed_local_path = local path where Canary will be saved after processing in Step 2 <br/>
biometrics_aws_path = S3 Key of Biometrics Source file in Step 1 <br/>
biometrics_local_path = local path where Biometrics will be saved after pulled from S3 in Step 1 <br/>
pmpm_aws_path = S3 Key of Original PMPM Source file in Step 1 <br/>
pmpm_local_path = local path where Original PMPM will be saved after pulled from S3 in Step 1 <br/>
pmpm_eligible_aws_path = S3 Key where processed PMPM is stored after Step 2 (for easy loading while experimenting, otherwise takes a long time) <br/>
pmpm_eligible_local_path = local path where processed PMPM is stored after pulling from S3 in Step 1 (if it was already processed before and saved in S3 for faster loading)

In [2]:
path_dict = {
    'data_path': '../../data',
    'temp_path': '../../temp',
    'aws_bucket':'cdphp-s3-us-e-p-pond',
    'canary_aws_path': 's3://cdphp-s3-us-e-p-pond/rpi/DataSources/PROGRAMS/',
    'canary_local_path': '../../data/CANARY',
    'canary_new_aws_path': 's3://cdphp-s3-us-e-p-pond/rpi/DataSources/FROM_POND/CA/Canary_detail_1220.parquet/',
    'canary_new_local_path': '../../data/CANARY_NEW',
    'canary_processed_local_path': '../../temp/CANARY.p',
    'biometrics_aws_path': 's3://cdphp-s3-us-e-p-pond/rpi/DataSources/BIOMETRICS202101/',
    'biometrics_local_path': '../../data/BIOMETRICS',
    'pmpm_aws_path': 's3://cdphp-s3-us-e-p-pond/rpi/DataSources/FROM_LAKE/PMPM/ANALYTIC_PMPM202101',
    'pmpm_local_path': '../../data/PMPM',
    'pmpm_eligible_aws_path': 'rpi/DATA_GEORGE/PMPM_PICKLE/ELIGIBLE.p',
    'pmpm_eligible_local_path': '../../temp/PMPM.p'
}

# 1. Download the Data
This function call will pull the data from S3 and save into the rpi-canary-eval/data folder and rpi-canary-eval/temp folders. The data folder contains, the original PMPM, Canary and Biometrics File. The temp folder contains/will contain the processed PMPM and Canary files. 

In [3]:
#notebooks/download_data/
data_process.download_data(path_dict)

Download Status: Success in Downloading Data
Total Data Download Time: 161.6367645263672


# 2. Process PMPM and Canary
Run Step 2.1 or 2.2, and then 2.3 <br/>
**2.1 PMPM Processing (Optional):** Processing PMPM is optional as it takes couple of hours every time. So for faster experiments, we have already processed the PMPM file once and saved it in S3 as ELIGIBLE.p file (look above in path_dict\['pmpm_eligible_aws_path'\]). <br/> If you want to do the PMPM processing (in case you are using new PMPM file), then process it. It will be saved both in S3 and local disk. <br/> 
Otherwise,if you have already processed a PMPM file and saved it in S3 and don't want to process now, you can just use that (you can skip step 2.1 and directly run step 2.2 which will download the processed file from S3 by specifying the path_dict\['pmpm_eligible_aws_path'\], path_dict\['pmpm_eligible_local_path'\] and path_dict\['aws_bucket'\]). <br/>

In [5]:
data_process.process_PMPM(path_dict)

**2.2 Processed PMPM load:** This step will load the processed PMPM file from S3 if PMPM was not processed in Step 2.1. If the processed file is already saved in '../../temp/' folder after Step 2.1, then it will not be reloaded from S3.

In [4]:
data_process.check_and_download_processed_PMPM(path_dict)

Download Status: Success in Downloading Data
Total Data Download Time: 60.94801378250122


**2.3 Canary Processing:** Process the Canary file and save it in the path specified in path_dict\['canary_processed_local_path'\]. You have to do this EVERYTIME. 

In [5]:
#notebooks/misc
data_process.process_canary(path_dict)

# 3. Load All the final datasets into program.
Processed Datasets are processed PMPM, processed Canary and Biometrics.

In [16]:
#notebooks/causal_data_creation/biometric_matching_X
canary, biometrics, pmpm = data_process.load_data()

# 4. Generate data by changing outcome and other parameters

**Step 4.1 Prepare Data file for any custome outcome here (Optional):** Generates outcome for diabetes onset (using PMPM), by generating a 'biometrics like' file for diabetes onset. Only need to do this if we are using outcomes outside of Biometrics file. <br/>
For diabetes onset, bigger_better=False, Outcome 1 is good, 0 is bad, 0 means x number of people transitioned from non-diabetes to diabetes (diabetes onset). <br/>
Run only if you want diabetes onset as outcome, otherwise comment this line

In [17]:
# biometrics = data_process.gen_diab_onset(pmpm)

**Step 4.2 Generate Final Data:** Generate the final dataset with features, outcomes, and treatment column. Change the outcome_name and other parameters as necessary.

In [18]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [19]:
%%time
#notebooks/causal_data_creation/biometric_matching_X
th = 6.6
merge_how = 'inner'
control_filters = 'new'
outcome_name = 'A1C'
df, treated_help =  data_functions.create_causal_data(canary.copy(), pmpm.copy(), biometrics.copy(), 
                      outcome_under_study = outcome_name, drop_single = True,
                      enroll_filt = True, enroll_months = 12, merge_how = merge_how,
                      max_days_apart = 180, min_days_apart = 120, threshold = th,
                                       bigger_better = False, window = 3,
                                      control_filters = control_filters)

Starting Control Outcomes New 0.33s per 100 id processing
Total of: 60489 ids, time to finish: 181.46699999999998 s
CPU times: user 4min 35s, sys: 11.2 s, total: 4min 47s
Wall time: 4min 46s


# 5. Save final data file
Save the final data file. It will be used for input to DoWhy Causal Inference package. 

In [20]:
#notebooks/causal_data_creation/biometric_matching_X
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df.to_pickle(f'../../matching_data/data_{outcome_name}.p')